# [Module 6.1] 추천 결과 분석

이 노트북을 실행하기 전에 먼저 "데이타 셋의 분리"에 대해서 다시 한번 보겠습니다.
Training Data (학습 데이터 셋)는 Solution Version을 학습하여 Campaign을 생성하였고 Validation(holdout, 검증 데이터 셋)은 이 노트북에서 Campaign을 통해 나온 추천 결과와 비교하여 성능 지표를 계산 합니다. Coldstart는 역시 이 노트북에서 Coldstart의 recipe통해 만든 Campaign의 성능을 확인 합니다.
<div>
<img src="static/imgs/img_datasplit50_v1.png", width="800">
</div>



이 노트북에서는 아래와 같은 작업을 합니다.
- 유저 개인별 추천 분석
    - **캠페인을 통해서 추천 결과가 나오면, 어떻게 해서 이런 추천 결과가 나올까 라는 호기심이 생깁니다.** 이런 호기심을 충족하기 위해서, (1) 학습 데이터 셋에서의 최근 인터랙션의 영화 리스트를 확인하고, (2) 검증 데이터 셋의 리스트를 확인 합니다. 이후 (3) 캠페인을 통한 추천 결과 리스틀플 확인 합니다. (3)의 추천 결과는 (1) 의 기반 위에서 나온 겻을 확인할 수 있고, (3) 과 (2)를 비교하여 추천 정확도를 확인 합니다.


- 캠페인을 통한 Validation(검증) 데이터 셋과 과 Predictor의 지표 분석 (전체 6040명) 
    - **실제로 캠페인을 통한 추천의 성능이 얼마나 잘 되었는지를 확인 합니다.**
    - 또한 이 결과를 이전에 솔류션 버전에서 생성된 성능 지표와 비교 합니다.


- ColdStart Campaign 성능 분석
    - **"ColdStart 의 성능이 정말 잘 나올까?"** 라는 호기심이 생깁니다. 1853개의 ColdStart의 아이템에서 랜덤으로 선택한 리스트와 ColdStart Campaign의 결과를 비교 합니다.


- ColdStart Campaign 의 추천 결과 유추
    - **추천 결과를 받았습니다. 그런데 어떻게 추천을 하지 라는 호기심이 생깁니다.** 결론적으로 학습에 사용된 item의 장르에 기반해서 ColdStart 추천을 받은 것을 확인할 수 있습니다.

---
이 노트북은 실행 시간이 약 3분 소요 됩니다. 하지만 분석된 결과를 보시려면 약 20분 정도 걸립니다.


In [16]:
import pandas as pd, numpy as np
import io
import scipy.sparse as ss
import json
import time
import os
import boto3
from metrics import ndcg_at_k, precision_at_k, mean_reciprocal_rank

from tqdm import notebook
from IPython.display import display, HTML

In [17]:
%store -r

In [18]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

### Helper 함수

In [20]:
def get_recentViews_richdata(user_id, df_rich, campaign_arn, item_meta, num_history, num_recommend):
    '''
    학습된 데이터(Warm_Train)에서 최신 리스트 및 추천 아이템을 제공 함
    '''
    history_items = df_rich[df_rich['USER_ID']==user_id].tail(num_history)
    
    rec_response = personalize_runtime.get_recommendations(
                campaignArn = campaign_arn,
                userId = str(user_id)
            )
    rec_items = [int(x['itemId']) for x in rec_response['itemList']]
    rec_items_movies = item_meta.set_index('ITEM_ID').loc[rec_items[:num_recommend]]

    return history_items, rec_items_movies

def get_user_relevance_list(campaign_arn, df_holdout, user_id):
    '''
    한명의 유저에 대해서 validation(warm_holdout)와 추천 리스트에서 매치된 것을 제공
    '''
    relevance = []

    true_items = set(df_holdout[df_holdout['USER_ID']==user_id]['ITEM_ID'].values)
    rec_response = personalize_runtime.get_recommendations(
        campaignArn = campaign_arn,
        userId = str(user_id)
    )
    rec_items = [int(x['itemId']) for x in rec_response['itemList']]
    relevance.append([int(x in true_items) for x in rec_items])
    return relevance


def get_relevance_list(campaign_arn, df_holdout, test_user_list):
    '''
    유저 그룹에 대해서 validation(warm_holdout)와 추천 리스트에서 매치된 것을 제공    
    '''
    relevance = []
    for user_id in test_user_list:
        true_items = set(df_holdout[df_holdout['USER_ID']==user_id]['ITEM_ID'].values)
        rec_response = personalize_runtime.get_recommendations(
            campaignArn = campaign_arn,
            userId = str(user_id)
        )
        rec_items = [int(x['itemId']) for x in rec_response['itemList']]
        relevance.append([int(x in true_items) for x in rec_items])
    return relevance


## 유저 개인별 추천 분석

특정 유저에 대해서 "학습에 사용한 데이터 셋" 에서의 최근 인터랙션, "검증에 사용한 데이터 셋"의 인터렉션 및 캠페인을 통한 나온 추천 결과를 상호 분석을 해봅니다.

In [21]:
from utils import get_rich_dataset

# 학습에 사용된 warm_train 데이터 셋 로딩
df_warm_train = pd.read_csv(warm_train_interaction_filename)
# Validation 인 holdout 데이터 셋 로딩
df_holdout = pd.read_csv(validation_interaction_filename)
# item 정보 로딩
item_meta = pd.read_csv('./ml-1m/movies.dat',sep='::', encoding='latin1',names=['ITEM_ID', 'TITLE', 'GENRE'],)
# 영황 타이틀, 장르를 포함한 정보 리턴
df_warm_train_rich = get_rich_dataset(df_warm_train, item_meta)
df_warm_train_rich = df_warm_train_rich.sort_values('TIMESTAMP').copy()
# 영황 타이틀, 장르를 포함한 정보 리턴
df_holdout_rich = get_rich_dataset(df_holdout, item_meta)
df_holdout_rich = df_holdout_rich.sort_values('TIMESTAMP').copy()

num_history = 5 # warm_train에서 최근 데이타 보여줄 수
num_recommend = 10 # 캠페인을 통한 추천에서 보여줄 수


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


#### help 함수

In [22]:
def track_user_ratings_recommend(user_id, campaign_arn):
    '''
    유저의 선택한 영화 부터 추천에 대한 리스트 제공
    '''
    # show history (warm_train): 학습한 데이타
    history_items, rec_items_movies = get_recentViews_richdata(user_id, df_warm_train_rich, campaign_arn, item_meta, num_history, num_recommend)
    display(HTML("<font><b>The lastest 5 movies the user rated (Warm-Train Dataset): </b></font>"))
    display(history_items)

    # 매치 리스트: validation (warm_holdout) and 캠페인에서의 추천 리스트
    relevance_list = get_user_relevance_list(campaign_arn, df_holdout, user_id)
    display(HTML("<font><b>The matched list between validation (Holdout Dataset)and recommendation: </b></font>"))
    display(relevance_list)

    # validation (warm_holdout)
    display(HTML("<font><b>The validation list (Holdout Dataset): </b></font>"))
    display(df_holdout_rich[df_holdout_rich.USER_ID == user_id].tail(100))
    # 캠페인에서의 추천 리스트
    display(HTML("<font><b>The recommended list by Campaign: </b></font>"))
    display(rec_items_movies)
    

#### User 1

유저가  **Beauty and the Beast --> Toy Story --> Tarzan 순으로 영화와 상호작용하고(학습 데이터셋)**, <br>
그 다음 상호 작용은  **Hunchback of Notre Dame --> Bug's Life --> Mulan (검증 데이터 셋)** 이었습니다. (시간의 순으로 학습 데이터 셋과 검증 데이터 셋으로 나누었습니다.)
<br>위의 학습 데이터셋 만을 사용하여 캠페인을 만들고 추천리스트를 받았습니다. <br>
**추천된 영화의 Top5는 Mulan, Bug's Life, Tarzan, Lion King, Cinderella 이었습니다. 추천 첫번째, 두번째가 검증셋에 존재했습니다.** <br>
**우연이라 생각할 정도로 추천을 정확하게 잘 했습니다.**

- 학습에 사용된 최신의 5개의 영화를 보면 Animation|Children의 장르의 영화에 점수를 매기었고, Beauty and the Beast --> Toy Story --> Tarzan 과 인터랙션 했습니다.
- 검증에 사용된 영화는 Hunchback of Notre Dame --> Bug's Life --> Mulan 이었습니다.
- 학습에 기반한 캠페인에서 추천한 영화의 Top5는 Mulan, Bug's Life, Tarzan, Lion King, Cinderella 이었습니다.
- 검증/추천이 매치된 리스트는 25개의 추천 중에 [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]] 이었습니다.
첫번째, 두번째로 추천한 영화가 모두 검증 데이터에 있기 때문에 [1,1] 로 체크 했습니다.

In [23]:
user_id = 1    
track_user_ratings_recommend(user_id,  user_personalization_campaign_arn)    

,USER_ID,ITEM_ID,TITLE,GENRE,TIMESTAMP,DATE
5027,1,594,Snow White and the Seven Dwarfs (1937),Animation|Children's|Musical,978302268,2000-12-31 22:37:48
16384,1,1545,Ponette (1996),Drama,978824139,2001-01-06 23:35:39
7281,1,595,Beauty and the Beast (1991),Animation|Children's|Musical,978824268,2001-01-06 23:37:48
20367,1,1,Toy Story (1995),Animation|Children's|Comedy,978824268,2001-01-06 23:37:48
11384,1,2687,Tarzan (1999),Animation|Children's,978824268,2001-01-06 23:37:48


[[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

,USER_ID,ITEM_ID,TITLE,GENRE,TIMESTAMP,DATE
166,1,783,"Hunchback of Notre Dame, The (1996)",Animation|Children's|Musical,978824291,2001-01-06 23:38:11
0,1,2355,"Bug's Life, A (1998)",Animation|Children's|Comedy,978824291,2001-01-06 23:38:11
101,1,1907,Mulan (1998),Animation|Children's,978824330,2001-01-06 23:38:50


,TITLE,GENRE
ITEM_ID,,
1907,Mulan (1998),Animation|Children's
2355,"Bug's Life, A (1998)",Animation|Children's|Comedy
2687,Tarzan (1999),Animation|Children's
364,"Lion King, The (1994)",Animation|Children's|Musical
1022,Cinderella (1950),Animation|Children's|Musical
2081,"Little Mermaid, The (1989)",Animation|Children's|Comedy|Musical|Romance
2085,101 Dalmatians (1961),Animation|Children's
2080,Lady and the Tramp (1955),Animation|Children's|Comedy|Musical|Romance
262,"Little Princess, A (1995)",Children's|Drama


#### User 2


유저가  **Demolition Man --> Star Wars: Episode I - The Phantom Menace --> Alien: Resurrection 순으로 영화와 상호작용하고(학습 데이터셋)**, <br>
그 다음 상호 작용은  **Bodyguard--> Twister --> Outbreak --> Broken Arrow --> Jackal --> Armageddon --> Lost World: Jurassic Park (검증 데이터 셋)** 이었습니다. (시간의 순으로 학습 데이터 셋과 검증 데이터 셋으로 나누었습니다.)
<br>위의 학습 데이터셋 만을 사용하여 캠페인을 만들고 추천리스트를 받았습니다. <br>
**추천된 영화의 Top5는 Arachnophobia, Star Trek: Insurrection, Demolition Man, Independence Day, Lost World: Jurassic Park 이었습니다. 
추천 다섯번째가 검증셋에 존재했습니다.** <br>
**다섯번째에 맞추었지만 학습 데이터 셋의 영화의 장르는 Action|Adventure|Sci-Fi 대다수였고, 추천한 영화도 비슷한 장르였고, 이후에 6, 7, 8 번째도 모두 맞추었습니다..**

- 학습에 사용된 최신의 5개의 영화를 보면 Action|Adventure|Sci-Fi의 장르의 영화에 점수를 매기었고, Demolition Man --> Star Wars: Episode I - The Phantom Menace --> Alien: Resurrection 과 인터랙션 했습니다.
- 검증에 사용된 영화는 Bodyguard--> Twister --> Outbreak --> Broken Arrow --> Jackal --> Armageddon --> Lost World: Jurassic Park 이었습니다.
- 학습에 기반한 캠페인에서 추천한 영화의 Top5는 Mulan, Bug's Life, Tarzan, Lion King, Cinderella 이었습니다.
- 검증/추천이 매치된 리스트는 25개의 추천 중에 [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 이었습니다.
다섯번째 ~ 여덟번째까지 추천한 영화가 모두 검증 데이터에 있기 때문에 [1,1,1,1] 로 체크 했습니다.

In [37]:
user_id = 2
track_user_ratings_recommend(user_id,  user_personalization_campaign_arn)    

,USER_ID,ITEM_ID,TITLE,GENRE,TIMESTAMP,DATE
35539,2,3107,Backdraft (1991),Action|Drama,978300002,2000-12-31 22:00:02
63832,2,1597,Conspiracy Theory (1997),Action|Mystery|Romance|Thriller,978300025,2000-12-31 22:00:25
50473,2,442,Demolition Man (1993),Action|Sci-Fi,978300025,2000-12-31 22:00:25
26815,2,2628,Star Wars: Episode I - The Phantom Menace (1999),Action|Adventure|Fantasy|Sci-Fi,978300051,2000-12-31 22:00:51
47355,2,1690,Alien: Resurrection (1997),Action|Horror|Sci-Fi,978300051,2000-12-31 22:00:51


[[0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

,USER_ID,ITEM_ID,TITLE,GENRE,TIMESTAMP,DATE
469,2,3257,"Bodyguard, The (1992)",Action|Drama|Romance|Thriller,978300073,2000-12-31 22:01:13
555,2,736,Twister (1996),Action|Adventure|Romance|Thriller,978300100,2000-12-31 22:01:40
268,2,292,Outbreak (1995),Action|Drama|Thriller,978300123,2000-12-31 22:02:03
675,2,95,Broken Arrow (1996),Action|Thriller,978300143,2000-12-31 22:02:23
215,2,1687,"Jackal, The (1997)",Action|Thriller,978300174,2000-12-31 22:02:54
734,2,1917,Armageddon (1998),Action|Adventure|Sci-Fi|Thriller,978300174,2000-12-31 22:02:54
339,2,1544,"Lost World: Jurassic Park, The (1997)",Action|Adventure|Sci-Fi|Thriller,978300174,2000-12-31 22:02:54


,TITLE,GENRE
ITEM_ID,,
2699,Arachnophobia (1990),Action|Comedy|Sci-Fi|Thriller
2393,Star Trek: Insurrection (1998),Action|Sci-Fi
442,Demolition Man (1993),Action|Sci-Fi
780,Independence Day (ID4) (1996),Action|Sci-Fi|War
1544,"Lost World: Jurassic Park, The (1997)",Action|Adventure|Sci-Fi|Thriller


#### User 5002

유저가 Congo (Action|Adventure|Mystery|Sci-Fi) --> Monty Python and the Holy Grail (Comedy) --> Rosencrantz and Guildenstern Are Dead (Comedy|Drama)--> What's Eating Gilbert Grape (Drama) --> Beauty and the Beast (Animation|Children's|Musical) 순으로 영화와 상호작용 했습니다.(학습 데이터셋)** <br>
**여기서 5번째 영화는 Beauty and the Beast (Animation|Children's|Musical) 로서 장르가 1-4번째 영화의 장르와 겹치지 않고 완전히 달랐습니다.** <br>
**추천된 리스트의 첫 번째 영화는 Bug's Life (Animation|Children's|Comedy) 로서 학습 데이터 셋의 1~4번째의 영화의 장르 대신에 5번째 영화에 해당하는 장르와 비슷한 영화를 추천하였습니다.**<br>
사용된 캠페인을 만들기 위한 user_personalization recipe가 시간의 흐름의 상호작용에  따라 추천을 해주는 것을 확인할 수 있습니다.



In [25]:
user_id = 5002
track_user_ratings_recommend(user_id,  user_personalization_campaign_arn)    

,USER_ID,ITEM_ID,TITLE,GENRE,TIMESTAMP,DATE
323058,5002,160,Congo (1995),Action|Adventure|Mystery|Sci-Fi,962604352,2000-07-03 06:05:52
91917,5002,1136,Monty Python and the Holy Grail (1974),Comedy,963026278,2000-07-08 03:17:58
125904,5002,1243,Rosencrantz and Guildenstern Are Dead (1990),Comedy|Drama,963027003,2000-07-08 03:30:03
152480,5002,337,What's Eating Gilbert Grape (1993),Drama,963027311,2000-07-08 03:35:11
8071,5002,595,Beauty and the Beast (1991),Animation|Children's|Musical,963027451,2000-07-08 03:37:31


[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]]

,USER_ID,ITEM_ID,TITLE,GENRE,TIMESTAMP,DATE
31308,5002,2160,Rosemary's Baby (1968),Horror|Thriller,963028123,2000-07-08 03:48:43
18429,5002,2791,Airplane! (1980),Comedy,963278308,2000-07-11 01:18:28
43474,5002,2416,Back to School (1986),Comedy,963278347,2000-07-11 01:19:07
29575,5002,674,Barbarella (1968),Adventure|Sci-Fi,963278381,2000-07-11 01:19:41
6893,5002,3255,"League of Their Own, A (1992)",Comedy|Drama,963281233,2000-07-11 02:07:13
38931,5002,3100,"River Runs Through It, A (1992)",Drama,963281305,2000-07-11 02:08:25
35582,5002,1029,Dumbo (1941),Animation|Children's|Musical,963281378,2000-07-11 02:09:38
12951,5002,708,"Truth About Cats & Dogs, The (1996)",Comedy|Romance,963281736,2000-07-11 02:15:36
22706,5002,1441,Benny & Joon (1993),Comedy|Romance,963281749,2000-07-11 02:15:49
31066,5002,383,Wyatt Earp (1994),Western,963282010,2000-07-11 02:20:10


,TITLE,GENRE
ITEM_ID,,
2355,"Bug's Life, A (1998)",Animation|Children's|Comedy
2791,Airplane! (1980),Comedy
1265,Groundhog Day (1993),Comedy|Romance
296,Pulp Fiction (1994),Crime|Drama
223,Clerks (1994),Comedy
1,Toy Story (1995),Animation|Children's|Comedy
2762,"Sixth Sense, The (1999)",Thriller
364,"Lion King, The (1994)",Animation|Children's|Musical
1517,Austin Powers: International Man of Mystery (1...,Comedy


## Validaton(holdout, 검증) 데이터 세트 평가하기

이번 파트에서는 앞장에 남겨두었던 데이터 세트를 활용하여 모델 성능을 평가 하도록 합니다.
테스트 데이터 셋에 있는 모든 고유한 사용자에 대해 테스트 데이터 세트 Interaction Ground Truth data와 Campaign에서 생성된 결과를 비교 하도록 합니다.

#### Help 함수

In [26]:

def evaluate_relevance(relevance):
    '''
    relevance 입력 받으면, 지표를 사전으로 리턴 함
    '''
    mrr = np.mean([mean_reciprocal_rank(r) for r in relevance])
    p_at_5= np.mean([precision_at_k(r, 5) for r in relevance])
    p_at_10 = np.mean([precision_at_k(r, 10) for r in relevance])
    p_at_25 = np.mean([precision_at_k(r, 25) for r in relevance])
    ndcg_at_5 = np.mean([ndcg_at_k(r, 5) for r in relevance])
    ndcg_at_10 = np.mean([ndcg_at_k(r, 10) for r in relevance])
    ndcg_at_25 = np.mean([ndcg_at_k(r, 25) for r in relevance])    
#     print('mean_reciprocal_rank: ',mrr)
#     print('precision_at_5: ',p_at_5)
#     print('precision_at_10: ',p_at_10)
#     print('precision_at_25: ', p_at_25)
#     print('normalized_discounted_cumulative_gain_at_5: ', ndcg_at_5)
#     print('normalized_discounted_cu{}mulative_gain_at_10: ',ndcg_at_10 )
#     print('normalized_discounted_cumulative_gain_at_25: ',ndcg_at_25 )    

    metric_dict = {}
    metric_dict['mrr'] = round(mrr,3)
    metric_dict['ndcg_at_5'] = round(ndcg_at_5,3)        
    metric_dict['ndcg_at_10'] = round(ndcg_at_10,3)            
    metric_dict['ndcg_at_25'] = round(ndcg_at_25,3)                
    metric_dict['p_at_5'] = round(p_at_5,3)    
    metric_dict['p_at_10'] = round(p_at_10, 3)
    metric_dict['p_at_25'] = round(p_at_25,3)    
        
    return metric_dict

def build_metric_matrix(solution,metric_dict):
    metrics.append([solution,
                        metric_dict['mrr'],
                        metric_dict['p_at_5'],
                        metric_dict['p_at_10'],
                        metric_dict['p_at_25'],
                        metric_dict['ndcg_at_5'],
                        metric_dict['ndcg_at_10'],
                        metric_dict['ndcg_at_25']

])


#### Validation 결과 확인 하기

아래 과정은 num_test_user = 6040 (전체) 으로 하면 약 25분 소요 됩니다.
디폴트로 10을 설정 합니다.

In [27]:
test_users = df_holdout['USER_ID'].unique()

# num_validation_test_user = 6040
num_validation_test_user = 10
test_user_list = test_users[:num_validation_test_user]

In [28]:
%%time

metrics=[] # 변수 선언
# user-perssonalization
relevance = get_relevance_list(user_personalization_campaign_arn, df_holdout, test_user_list)
metrics_eval_rel = evaluate_relevance(relevance)
build_metric_matrix("user-pers",metrics_eval_rel)
# hrnn
relevance = get_relevance_list(hrnn_campaign_arn, df_holdout, test_user_list)
metrics_eval_rel = evaluate_relevance(relevance)
build_metric_matrix("hrnn",metrics_eval_rel)
# hrnn-meta
relevance = get_relevance_list(hrnn_meta_campaign_arn, df_holdout, test_user_list)
metrics_eval_rel = evaluate_relevance(relevance)
build_metric_matrix("hrnn-meta",metrics_eval_rel)
# hrnn-coldstart
relevance = get_relevance_list(hrnn_coldstart_campaign_arn, df_holdout, test_user_list)
metrics_eval_rel = evaluate_relevance(relevance)
build_metric_matrix("hrnn_coldstart",metrics_eval_rel)

CPU times: user 111 ms, sys: 7.5 ms, total: 118 ms
Wall time: 3.67 s


In [29]:
val_metrics = pd.DataFrame(metrics, 
                           columns=['recipe','mrr','ncdg@5','ncdg@10','ncdg@25','p@5','p@10','p@25'])
val_metrics

,recipe,mrr,ncdg@5,ncdg@10,ncdg@25,p@5,p@10,p@25
0,user-pers,0.215,0.08,0.09,0.068,0.138,0.222,0.338
1,hrnn,0.104,0.02,0.04,0.024,0.100,0.197,0.230
2,hrnn-meta,0.121,0.04,0.05,0.056,0.078,0.123,0.244
3,hrnn_coldstart,0.025,0.02,0.01,0.004,0.050,0.050,0.050


## 캠페인을 통한 Validation(검증) 데이터 셋과 과 솔류션 버전의 지표 분석




#### Validation 데이터 셋 (6040명)과 Campaign의 추천 결과를 비교하여 만든 평가 지표

검증 데이터 셋 에 대해서 아래와 같은 점을 확인할 수 있습니다.
- 4개의 레서피 중에서는 user_personalization이 가장 좋은 성능을 보였습니다. 그 다음은 hrnn-meta가 두번째로 좋은 성능을 보였습니다.
- user_personalization의 성능 지표를 설명하면
    - "mrr: 0.277은 추천된 리스트에서 약 4번째의 영화가 검증 셋에 존재한다" 라고 해석이 됩니다. (1/4 = 0.25)
    - "ncdg@5:0.114는 가장 이상적인 상황인 1.0 (예: 검증셋[영화A,영화E,영화T], 추천[영화A,영화E,영화T,영화S, 영화U]) 대비 약 11% 정도의 성능이다"라고 해석할 수 있습니다.
    - "p@5: 0.209" 는 추천한 영화가 5개이면 이중에 약 1개가 (1/5 = 0.2) 검증셋에 있다"라고 해석 됩니다.
- item_meta를 사용하지 않은 hrnn 보다 item_meta를 사용하는 hrnn_meta가 성능이 좋습니다.
- hrnn_coldstart의 경우에는 데이타의 구성상 성능이 나올 수 없습니다. item_meta 테이블에
1853 개는 (1) 학습/검증 데이터 셋에 포함이 되어 있고, 1853개는 (2)coldstart 데이터 셋에 포함되어 있습니다. 즉 (2) 에서 coldstart recipe가 아이템을 찾아 주로 추천을 하는데, (1)에 있는 검증 데이터 셋에 있는 아이템이  전혀 (2)에 없기에 성능이 나올 수 없습니다.

<div>
<img src="static/imgs/Fig.6.1.validation_metric_summary.png" width="800"/>
</div>

#### 학습 데이터 셋 (6040명) 통한 솔류션 버전의 평가 지표
Predictors를 통해서 나온 지표가 "Validation 데이터 셋 (6040명)과 Campaign의 추천 결과" 대비 지표마다 차이가 있습니다. ncdg가 전반적으로 더 좋게 나왔습니다.
<div>
<img src="static/imgs/Fig.3.2.metric_summary.png" width="800"/>
</div>



## Cold Start 성능 테스트 

이부분에서는 새로운 아이템(ColdStart)에 대한 추천 성능을 테스트 해보도록 합니다. <br>
아래는 소요식간 상 10명으로 했습니다. 
하지만 아래의 결과 분석은 총 유저수인 6040명으로 실행하여 분석 하였습니다.


In [30]:
df_coldstart=pd.read_csv(coldstart_interation_filename)
test_users = df_coldstart['USER_ID'].unique()
# num_coldstart_test_user = 6040
num_coldstart_test_user = 10
test_user_list = test_users[:num_coldstart_test_user]

In [31]:
metrics=[] # 변수 선언

# hrnn-coldstart
relevance = get_relevance_list(hrnn_coldstart_campaign_arn, df_coldstart, test_user_list)
metrics_eval_rel = evaluate_relevance(relevance)
build_metric_matrix("hrnn_coldstart",metrics_eval_rel)
# random
def get_random_relevance_list(df_coldstart, test_user_list):
    relevance = []
    for user_id in  test_user_list:
        true_items = set(df_coldstart[df_coldstart['USER_ID']==user_id]['ITEM_ID'].values)
        rec_items = np.random.RandomState(seed=42).permutation(cold_items)[:25]
        relevance.append([int(x in true_items) for x in rec_items])
        
    return relevance

relevance = get_random_relevance_list(df_coldstart, test_user_list)
metrics_eval_rel = evaluate_relevance(relevance)
build_metric_matrix("random",metrics_eval_rel)

In [32]:
val_metrics = pd.DataFrame(metrics, columns=['recipe','mrr','ncdg@5','ncdg@10','ncdg@25','p@5','p@10','p@25'])
val_metrics

,recipe,mrr,ncdg@5,ncdg@10,ncdg@25,p@5,p@10,p@25
0,hrnn_coldstart,0.320,0.14,0.14,0.104,0.216,0.274,0.403
1,random,0.105,0.04,0.04,0.020,0.113,0.168,0.190


## Coldstart 와 Random 추천의 성능 비교 분석(6040명)
HRNN Cold Start모델은 메타 데이터의 일부 정보를 활용하여 Interaction정보가 없는 새로운 아이템에 대해도 추천을 할수 있습니다.<br>
메타 데이터 정보가 장르밖에 없었음에도 랜덤 추천 대비 약 3~4배의 성능이 있었음을 확인할 수 있습니다. <br>
메타 데이터 성능을 향상 시키거나 Cold-start item비율을 줄인다면 더 좋은 성능을 기대해 볼 수 있습니다. 

<div>
<img src="static/imgs/Fig.6.1.coldstart_random_metrics.png" width="800"/>
</div>




## ColdStart Recipe 추천 결과의 유추

In [33]:
from utils import get_rich_dataset
df_warm_train = pd.read_csv(warm_train_interaction_filename)
item_meta = pd.read_csv('./ml-1m/movies.dat',sep='::', encoding='latin1',names=['ITEM_ID', 'TITLE', 'GENRE'],)
df_warm_train_rich = get_rich_dataset(df_warm_train, item_meta)
df_warm_train_rich = df_warm_train_rich.sort_values('TIMESTAMP').copy()

num_history = 5
num_recommend = 5


def show_coldstart_effect(user_id):
    user_id = df_coldstart.USER_ID.unique()[coldstart_test_user]

    history_items , rec_items_movies= get_recentViews_richdata(user_id, df_warm_train_rich, hrnn_coldstart_campaign_arn, item_meta, num_history, num_recommend)

    display(HTML("<font><b>The lastest top 5 movies fed into learned model (Warm-Train Dataset): </b></font>"))
    display(history_items)
    display(HTML("<font><b>Five movies recommended by ColdStart recipe: </b></font>"))
    display(rec_items_movies)
    


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  app.launch_new_instance()


#### User 2

**이 사용자는액션|어드벤처|스릴러 아이템을 많이 선택하였고 <br>
ColdStart Recipe를 통한 추천도 비슷한 장르에서 만이 선택하였다는 것을 다는 것을 알았습니다.** <br>
콜드 아이템에서 액션|어드벤처|스릴러 아이템을 추천하고 있습니다.

In [34]:
coldstart_test_user = 1    
show_coldstart_effect(coldstart_test_user)    

,USER_ID,ITEM_ID,TITLE,GENRE,TIMESTAMP,DATE
35539,2,3107,Backdraft (1991),Action|Drama,978300002,2000-12-31 22:00:02
63832,2,1597,Conspiracy Theory (1997),Action|Mystery|Romance|Thriller,978300025,2000-12-31 22:00:25
50473,2,442,Demolition Man (1993),Action|Sci-Fi,978300025,2000-12-31 22:00:25
26815,2,2628,Star Wars: Episode I - The Phantom Menace (1999),Action|Adventure|Fantasy|Sci-Fi,978300051,2000-12-31 22:00:51
47355,2,1690,Alien: Resurrection (1997),Action|Horror|Sci-Fi,978300051,2000-12-31 22:00:51


,TITLE,GENRE
ITEM_ID,,
1205,"Transformers: The Movie, The (1986)",Action|Animation|Children's|Sci-Fi|Thriller|War
610,Heavy Metal (1981),Action|Adventure|Animation|Horror|Sci-Fi
849,Escape from L.A. (1996),Action|Adventure|Sci-Fi|Thriller
1917,Armageddon (1998),Action|Adventure|Sci-Fi|Thriller
1591,Spawn (1997),Action|Adventure|Sci-Fi|Thriller


#### 유저 3

**이 사용자는  Animation|Comedy|Children 아이템을 많이 선택하였고 <br>
ColdStart Recipe를 통한 추천도 비슷한 장르에서 만이 선택하였다는 것을 다는 것을 알았습니다.** <br>
콜드 아이템에서 Animation|Comedy|Children 아이템을 추천하고 있습니다.


In [35]:
coldstart_test_user = 2 
show_coldstart_effect(coldstart_test_user)    

,USER_ID,ITEM_ID,TITLE,GENRE,TIMESTAMP,DATE
90723,3,1136,Monty Python and the Holy Grail (1974),Comedy,978298079,2000-12-31 21:27:59
24236,3,3114,Toy Story 2 (1999),Animation|Children's|Comedy,978298103,2000-12-31 21:28:23
84384,3,3619,"Hollywood Knights, The (1980)",Comedy,978298201,2000-12-31 21:30:01
53307,3,1265,Groundhog Day (1993),Comedy|Romance,978298316,2000-12-31 21:31:56
84948,3,2355,"Bug's Life, A (1998)",Animation|Children's|Comedy,978298430,2000-12-31 21:33:50


,TITLE,GENRE
ITEM_ID,,
1566,Hercules (1997),Adventure|Animation|Children's|Comedy|Musical
588,Aladdin (1992),Animation|Children's|Comedy|Musical
2078,"Jungle Book, The (1967)",Animation|Children's|Comedy|Musical
239,"Goofy Movie, A (1995)",Animation|Children's|Comedy|Romance
2141,"American Tail, An (1986)",Animation|Children's|Comedy


#### 유저 500

**이 사용자는  Drama장르의 아이템을 많이 선택하였고 <br>
ColdStart Recipe를 통한 추천도 비슷한 장르에서 만이 선택하였다는 것을 다는 것을 알았습니다.** <br>
콜드 아이템에서 Drama|Romance|Comedy 아이템을 추천하고 있습니다.


In [36]:
coldstart_test_user = 499   
show_coldstart_effect(coldstart_test_user)    

,USER_ID,ITEM_ID,TITLE,GENRE,TIMESTAMP,DATE
124126,500,497,Much Ado About Nothing (1993),Comedy|Romance,976644137,2000-12-12 18:02:17
152018,500,337,What's Eating Gilbert Grape (1993),Drama,976644171,2000-12-12 18:02:51
17239,500,2762,"Sixth Sense, The (1999)",Thriller,976644186,2000-12-12 18:03:06
1132,500,3408,Erin Brockovich (2000),Drama,979257574,2001-01-11 23:59:34
408772,500,3795,"Five Senses, The (1999)",Drama,979257748,2001-01-12 00:02:28


,TITLE,GENRE
ITEM_ID,,
1094,"Crying Game, The (1992)",Drama|Romance|War
1944,From Here to Eternity (1953),Drama|Romance|War
912,Casablanca (1942),Drama|Romance|War
224,Don Juan DeMarco (1995),Comedy|Drama|Romance
3343,And God Created Woman (1988),Comedy|Drama|Romance
